# Build Models with TensorFlow 2.x APIs
![simple_nn](media/miscellaneous/tf_logo.png "TF Logo")

In [1]:
# import tensorflow 2.x
import tensorflow as tf
# or
from tensorflow import keras

# 1.0 Sequential API

Single stack of layers connected sequentially

![simple_nn](media/tf_tutorial_1/mlp.png "Classical MLP")

## 1.1 First sequential method

In [2]:
model = tf.keras.models.Sequential()

In [3]:
model.add(tf.keras.layers.Dense(9, input_shape=[8], activation='relu', name='hidden_layer_1'))
for i in range(2):
    model.add(tf.keras.layers.Dense(9, activation='relu', name=f'hidden_layer_{i+2}')) # or tf.kerars.activations.relu
     # or tf.keras.layers.ReLU() as layer
model.add(tf.keras.layers.Dense(4, activation='softmax', name='output_layer')) # supposing is a classification task

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden_layer_1 (Dense)       (None, 9)                 81        
_________________________________________________________________
hidden_layer_2 (Dense)       (None, 9)                 90        
_________________________________________________________________
hidden_layer_3 (Dense)       (None, 9)                 90        
_________________________________________________________________
output_layer (Dense)         (None, 4)                 40        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [5]:
((8 * 9) + 9) + ((9 * 9) + 9) + ((9 * 9) + 9) + ((9 * 4) + 4) # W 'n' b

301

## 1.2 Second sequential method

In [6]:
model_seq = tf.keras.models.Sequential([
    tf.keras.layers.Dense(9, input_shape=[8], activation='relu', name='hidden_layer_1'),
    tf.keras.layers.Dense(9, activation='relu', name='hidden_layer_2'),
    tf.keras.layers.Dense(9, activation='relu', name='hidden_layer_3'),
    tf.keras.layers.Dense(4, activation='softmax', name='output_layer')  
])

## 1.3 Access models layers 

In [7]:
model.layers # it's a list

In [8]:
model.layers[0] # by index

In [9]:
model.layers[0].name # name model

'hidden_layer_1'

In [10]:
model.get_layer('hidden_layer_1') # by name

In [11]:
model.layers[0].get_weights() # also set weights

[array([[-0.20320478, -0.21385965,  0.46539867, -0.565007  , -0.2593493 ,
         -0.097994  , -0.32468057,  0.5229882 ,  0.182953  ],
        [ 0.24967742,  0.03368342, -0.161809  ,  0.4860531 ,  0.17167604,
          0.3537783 ,  0.50933146,  0.12967998, -0.11753729],
        [-0.35103965,  0.5018636 , -0.08083469, -0.20250365, -0.31604254,
         -0.47069424, -0.15943393,  0.4115677 , -0.32681623],
        [ 0.1673544 , -0.55576795,  0.02488774, -0.2572023 ,  0.38952065,
         -0.4156562 ,  0.37581748,  0.327856  , -0.2215968 ],
        [-0.3132342 , -0.0796566 , -0.3469519 ,  0.25162995,  0.4238857 ,
         -0.05401027,  0.03864241,  0.01954091,  0.07278502],
        [ 0.43403828,  0.45341873, -0.5102984 ,  0.22954148,  0.33059877,
          0.38376117, -0.02830613,  0.3519755 , -0.3106324 ],
        [ 0.04033619,  0.11887825,  0.55660677, -0.5476306 ,  0.47074115,
         -0.15201232,  0.04258358, -0.57668024, -0.04115087],
        [-0.21281064, -0.5757724 , -0.5558327 , 

# 2.0 Functional API

Build networks with complex topologies or with multiple inputs and outputs

## 2.1 Build a network block

Example with inception block

<p align="center">
<img src="media/tf_tutorial_1/inception_block.png" width="550px">
</p>

In [12]:
def inception_block(input_previous_layer, filters):
    # first branch
    x_1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=(1,1), 
                                 strides=(1, 1), padding='same', activation='relu')(input_previous_layer)
    # second branch
    x_2_bn = tf.keras.layers.Conv2D(filters=filters//2, kernel_size=(1,1), 
                                 strides=(1, 1), padding='same', activation='relu')(input_previous_layer)
    x_2 = tf.keras.layers.Conv2D(filters=filters, kernel_size=(3,3), 
                                 strides=(1, 1), padding='same', activation='relu')(x_2_bn)
    # third branch
    x_3_bn = tf.keras.layers.Conv2D(filters=filters//2, kernel_size=(1,1), 
                                 strides=(1, 1), padding='same', activation='relu')(input_previous_layer)
    x_3 = tf.keras.layers.Conv2D(filters=filters, kernel_size=(5,5), 
                                 strides=(1, 1), padding='same', activation='relu')(x_3_bn)
    
    # fourth branch
    x_4_max = tf.keras.layers.MaxPool2D(pool_size=(3, 3), padding='same')(input_previous_layer)
    x_4 = tf.keras.layers.Conv2D(filters=filters, kernel_size=(1,1), 
                                 strides=(1, 1), padding='same', activation='relu')(x_4_max)
    
    # output
    x_output = tf.keras.layers.Concatenate()([x_1, x_2, x_3, x_4])
    
    return x_output

## 2.2 Build a model

<p align="center">
<img src="media/tf_tutorial_1/func_api_example.png" width="650px">
</p>

In [13]:
def build_model(N):
    x_input = tf.keras.layers.Input(shape=(None,None,3))
    
    # residual
    x_res = x_input
    
    # N inception blocks
    x = inception_block(x_input, filters=16)
    for i in range(N):
        x = inception_block(x, filters=32)
        
    # convolution 1x1
    x_conv_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(1,1), 
                                 strides=(1, 1), padding='same', activation='relu')(x)        
    # convolution 3x3
    x_conv_3 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), 
                                 strides=(1, 1), padding='same', activation='relu')(x)
    
    # concatenate
    x_output = tf.keras.layers.Concatenate()([x_conv_1, x_conv_3, x_res])
    
    return tf.keras.Model(inputs=x_input, outputs=x_output, name='function_api_test')

In [14]:
model_f_api = build_model(N=3)

In [15]:
model_f_api.summary(line_length=120)

Model: "function_api_test"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_1 (InputLayer)                   [(None, None, None, 3)]    0                                                     
________________________________________________________________________________________________________________________
conv2d_1 (Conv2D)                      (None, None, None, 8)      32            input_1[0][0]                           
________________________________________________________________________________________________________________________
conv2d_3 (Conv2D)                      (None, None, None, 8)      32            input_1[0][0]                           
________________________________________________________________________________________________________________________
max_p

## 2.3 Multiple inputs and outputs

<p align="center">
<img src="media/tf_tutorial_1/func_api_example2.png" width="650px">
</p>

In [16]:
def build_model_mult(N):
    x_input_1 = tf.keras.layers.Input(shape=(None,None,3))
    x_input_2 = tf.keras.layers.Input(shape=(None,None,3))
    
    
    # N inception blocks
    x = inception_block(x_input_1, filters=16)
    for i in range(N):
        x = inception_block(x, filters=32)
        
    # convolution 1x1
    x_conv_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(1,1), 
                                 strides=(1, 1), padding='same', activation='relu')(x)        
    # convolution 3x3
    x_conv_3 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), 
                                 strides=(1, 1), padding='same', activation='relu')(x)
    
    # concatenate
    x_output_1 = tf.keras.layers.Concatenate()([x_conv_1, x_conv_3, x_input_2])
    x_output_2 = x_conv_3
    
    return tf.keras.Model(inputs=[x_input_1, x_input_2], outputs=[x_output_1, x_output_2], name='function_api_test')

In [17]:
model = build_model_mult(N=3)

In [18]:
model.summary(line_length=120)

Model: "function_api_test"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_2 (InputLayer)                   [(None, None, None, 3)]    0                                                     
________________________________________________________________________________________________________________________
conv2d_27 (Conv2D)                     (None, None, None, 8)      32            input_2[0][0]                           
________________________________________________________________________________________________________________________
conv2d_29 (Conv2D)                     (None, None, None, 8)      32            input_2[0][0]                           
________________________________________________________________________________________________________________________
max_p

## 2.4 Combine models

In [19]:
def build_combine_model(model_1, model_2):
    x_input = tf.keras.layers.Input(shape=(None,None,3))
    
    # feed first model
    x = model_1(x_input)
    
    # process outputs first model
    x = tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), 
                                 strides=(1, 1), padding='same', activation='relu')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    
    # feed second model
    x_output = model_2(x)
    
    return tf.keras.Model(inputs=x_input, outputs=x_output, name='combine_models_test')

In [20]:
combined_model = build_combine_model(model_f_api, model_seq)

In [21]:
combined_model.summary(line_length=120)

Model: "combine_models_test"
________________________________________________________________________________________________________________________
Layer (type)                                          Output Shape                                    Param #           
input_4 (InputLayer)                                  [(None, None, None, 3)]                         0                 
________________________________________________________________________________________________________________________
function_api_test (Model)                             (None, None, None, 67)                          129024            
________________________________________________________________________________________________________________________
conv2d_52 (Conv2D)                                    (None, None, None, 8)                           4832              
________________________________________________________________________________________________________________________
glo

# 3.0 Subclassing API

Sequential and Functional API require to define layers and their connections in a static manner. Once a grarph is declared can be used for training and inferernce. However, that could be a limitation for some projects.

**Advantages of Subclassing API**: 
- build dynamic models that can deal with varying shapes, loops and conditional branches
- more imperative programming style

**Disadvantages of Subclassing API**:
- it's not simple to save and clone a model
- summary() method shows only a list of layers and not how they are connected
- more prone to errors and more difficult to debug and inspect

## 3.1 First dumb example

![simple_nn](media/tf_tutorial_1/mlp.png "Classical MLP")

In [22]:
class MyModel(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(9, activation='relu')
        self.hidden2 = tf.keras.layers.Dense(9, activation='relu')
        self.hidden3 = tf.keras.layers.Dense(9, activation='relu')
        self.output_layer = tf.keras.layers.Dense(4, activation='softmax')
        
    def call(self, inputs):
        x = self.hidden1(inputs)
        x = self.hidden2(x)        
        x = self.hidden3(x)
        outputs = self.output_layer(x)
        
        return outputs

In [23]:
model = MyModel(name='my_sub_mlp')

In [24]:
model.build(input_shape=(32,8))

In [25]:
model.summary()

Model: "my_sub_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  81        
_________________________________________________________________
dense_1 (Dense)              multiple                  90        
_________________________________________________________________
dense_2 (Dense)              multiple                  90        
_________________________________________________________________
dense_3 (Dense)              multiple                  40        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


## 3.2 Second less dumb example

<p align="center">
<img src="media/tf_tutorial_1/resnet_full.png" width="950px">
</p>

### Build the ResNet block

<p align="center">
<img src="media/tf_tutorial_1/resnet_block.png" width="550px">
</p>

In [26]:
class ResNetBlock(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.identity_block = [
                     tf.keras.layers.Conv2D(filters//4, 1, strides=strides, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     self.activation,
                     tf.keras.layers.Conv2D(filters//4, 3, strides=1, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     self.activation,
                     tf.keras.layers.Conv2D(filters, 1, strides=1, padding='same'),
                     tf.keras.layers.BatchNormalization(),
                     self.activation]
        self.conv_block = []
        if strides > 1:
            self.conv_block = [
                     tf.keras.layers.Conv2D(filters, 1, strides=strides, padding='same'),
                     tf.keras.layers.BatchNormalization()]
    def call(self, inputs):
        x = inputs
        for layer in self.identity_block:
            x = layer(x)
        x_res = inputs
        for layer in self.conv_block:
            x_res = layer(x_res)
        return self.activation(x + x_res)

### Build the network

<p align="center">
<img src="media/tf_tutorial_1/resnet_architectures.png" width="750px">
</p>

In [27]:
class ResNet(tf.keras.Model):

    def __init__(self, n_block1, n_block2, n_block3, n_block4, num_classes, activation='relu'):
        super(ResNet, self).__init__()
        # initial layers
        self.zerop1 = tf.keras.layers.ZeroPadding2D(padding=(3, 3))
        self.zerop2 = tf.keras.layers.ZeroPadding2D(padding=(1, 1))
        self.activation = tf.keras.activations.get(activation)
        self.conv1 = tf.keras.layers.Conv2D(64, 7, strides=2)
        self.batch1 = tf.keras.layers.BatchNormalization()
        self.max1 = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=2, padding='valid')

        # residual blocks
        self.block_1 = [ResNetBlock(256) if i is not 0 else ResNetBlock(256, strides=2) for i in range(n_block1+1)]
        self.block_2 = [ResNetBlock(512) if i is not 0 else ResNetBlock(512, strides=2) for i in range(n_block2+1)]
        self.block_3 = [ResNetBlock(1024) if i is not 0 else ResNetBlock(1024, strides=2) for i in range(n_block3+1)]
        self.block_4 = [ResNetBlock(2048) if i is not 0 else ResNetBlock(2048, strides=2) for i in range(n_block4+1)]
        
        # final layers
        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(num_classes)

    def call(self, inputs):
        x = self.zerop1(inputs)
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.activation(x)
        x = self.zerop2(x)
        x = self.max1(x)
        
        for layer in self.block_1 + self.block_2 + self.block_3 + self.block_4:
            x = layer(x)
        
        x = self.global_pool(x)
        
        return self.classifier(x)

In [28]:
model = ResNet(3, 4, 6, 3, 1000)

In [29]:
model.build(input_shape=(32,224,224,3))

In [30]:
model.summary()

Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 multiple                  0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin multiple                  0         
_________________________________________________________________
conv2d_53 (Conv2D)           multiple                  9472      
_________________________________________________________________
batch_normalization (BatchNo multiple                  256       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 multiple                  0         
_________________________________________________________________
res_net_block (ResNetBlock)  multiple                  76928     
_________________________________________________________________
res_net_block_1 (ResNetBlock multiple                  7155